In [1]:
import numpy as np
from PIL import Image

def load_png_as_array(filepath):
    """Load a .png image and return its pixel values as a numpy array."""
    img = Image.open(filepath)
    arr = np.array(img)
    return arr

# Example usage:
# file_path = "path/to/your/image.png"
# arr = load_png_as_array(file_path)
# print(arr)


In [2]:
file_path = "/home/mliu/goldsgym/TubeletGraph/assets/example/0334_cut_fruit_1_0000000.png"

In [3]:
arr = load_png_as_array(file_path)

In [4]:
val_lyst = []
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        val_lyst.append(arr[i, j])

In [5]:
from collections import Counter
counts = Counter(val_lyst)
print(counts)

Counter({0: 882346, 1: 39254})


In [7]:
import os

def convert_mask_images_to_binary(input_dir, output_dir):
    """
    Converts all PNG mask images in input_dir (masked regions are #FF0000)
    to indexed PNG images (mode 'P') in output_dir.
    Masked region -> index 1 (color: 128,0,0), background -> index 0 (color: 0,0,0).
    Output images will be indexed PNG format compatible with load_anno().
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for fname in os.listdir(input_dir):
        if fname.lower().endswith(".png"):
            input_path = os.path.join(input_dir, fname)
            img = Image.open(input_path).convert("RGB")
            img_np = np.array(img)
            # Mask: red is (255,0,0)
            mask = (img_np[:,:,0] == 255) & (img_np[:,:,1] == 0) & (img_np[:,:,2] == 0)
            # Create indexed array: 0 = background, 1 = mask
            indexed_img = np.zeros((img_np.shape[0], img_np.shape[1]), dtype=np.uint8)
            indexed_img[mask] = 1
            
            # Create palette: index 0 = black (0,0,0), index 1 = dark red (128,0,0)
            palette = [0, 0, 0,    # index 0: black
                      128, 0, 0]   # index 1: dark red
            # Pad palette to 256 colors (768 values: 256 * 3)
            palette.extend([0] * (768 - len(palette)))
            
            # Create indexed image with palette
            mask_img_pil = Image.fromarray(indexed_img, mode='P')
            mask_img_pil.putpalette(palette)
            
            output_path = os.path.join(output_dir, fname)
            mask_img_pil.save(output_path)
            print(f"Converted: {fname} -> {output_path} (mode: {mask_img_pil.mode})")

# Example usage:
# convert_mask_images_to_binary(
#     input_dir="assets/example/masks",
#     output_dir="assets/example/masks_binary"
# )



In [8]:
convert_mask_images_to_binary("/home/mliu/goldsgym/TubeletGraph/assets/example/mask_rgb", "/home/mliu/goldsgym/TubeletGraph/assets/example/mask")

Converted: sam3_drawer2.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_drawer2.png (mode: P)
Converted: sam3_pottery_bowl.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_pottery_bowl.png (mode: P)
Converted: sam3_flower1.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_flower1.png (mode: P)
Converted: sam3_flower2.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_flower2.png (mode: P)
Converted: sam3_drawer1.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_drawer1.png (mode: P)
Converted: sam3_dumpling.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_dumpling.png (mode: P)
Converted: sam3_towel.png -> /home/mliu/goldsgym/TubeletGraph/assets/example/mask/sam3_towel.png (mode: P)


In [16]:
from PIL import Image
import numpy as np

img_path = "/home/mliu/goldsgym/TubeletGraph/assets/example/0334_cut_fruit_1_0000000.png"
img = Image.open(img_path).convert("RGB")
img_np = np.array(img)

# Print shape, dtype, and a few pixels
print(f"Shape: {img_np.shape}, dtype: {img_np.dtype}")

# Check if there are any red pixels (255,0,0)
red_mask = (img_np[:,:,0]==255) & (img_np[:,:,1]==0) & (img_np[:,:,2]==0)
print(f"Red pixel count: {np.sum(red_mask)}")

# Print a few pixel values from corners and center
h, w, _ = img_np.shape
sample_coords = [(0,0), (0,w//2), (h//2,0), (h//2,w//2), (h-1,w-1)]
print("Sampled pixel values (RGB):")
for y, x in sample_coords:
    print(f"({y},{x}): {img_np[y,x]}")

# Also, check unique RGB values in the image
unique_colors = np.unique(img_np.reshape(-1, 3), axis=0)
print(f"Unique RGB colors in mask image: {unique_colors}")

# If you want to visualize the red mask overlaid, you may save an overlay
# overlay = img_np.copy()
# overlay[red_mask] = [0,255,0]  # make red pixels green for visualization
# Image.fromarray(overlay).save("mask_red_overlay_vis.png")


Shape: (720, 1280, 3), dtype: uint8
Red pixel count: 0
Sampled pixel values (RGB):
(0,0): [0 0 0]
(0,640): [0 0 0]
(360,0): [0 0 0]
(360,640): [0 0 0]
(719,1279): [0 0 0]
Unique RGB colors in mask image: [[  0   0   0]
 [128   0   0]]
